In [1]:
from src.AlphabetPermutation import AlphabetPermutation
from src.Probability import ProbabilityMatrix
from string import ascii_lowercase
import numpy as np
import random

In [2]:
alphabet = list(ascii_lowercase)
alphabet.append(' ')

# Initialize an instance of permutation generator class
permutation_generator = AlphabetPermutation(alphabet)

# Obtain the key: in a cipher the key is the alphabet used by the cipher
key = permutation_generator.permutation.values()

In [3]:
message = "Good afternoon markov chain monte carlo let us see if you are any good at decrypting the text that I will provide for you to try out I will make it as long as possible so that you can try it out and get more chance to get it right"

In [4]:
encrypted_message = permutation_generator.permute_string(message)
encrypted_message

'aeeo puicbsees gpbkeq mzpvs gesic mpbde dci fw wcc vu tef pbc pst aeeo pi ocmbtrivsa izc icyi izpi v nvdd rbeqvoc ueb tef ie ibt efi v nvdd gpkc vi pw desa pw rewwvldc we izpi tef mps ibt vi efi pso aci gebc mzpsmc ie aci vi bvazi'

In [99]:
"""
To decrypt BACK

d = dict.fromkeys(list(permutation_generator.permutation.keys()))
d = dict(zip(list(permutation_generator.permutation.values()), list(permutation_generator.permutation.keys())))
permutation_generator.permutation = d
permutation_generator.permute_string(encrypted_message)
""";

In [5]:
with open("texts/shakespeare.txt", "r") as file:
    text = file.read()
alphabet = list("abcdefghijklmnopqrstuvwxyz ")

In [6]:
pm = ProbabilityMatrix(text=text, alphabet=alphabet)
unknown_chars = pm.unknown_chars()
pm.preprocess_text(unknown_chars=unknown_chars)

pm.compute_matrix_spaces()
log_matrix = pm.compute_log_matrix()

In [7]:
def compute_likelihood(encrypted_message, log_transition_matrix):
    # Add whitespace before the message
    encrypted_message = " " + encrypted_message

    log_lik = 0
    for i in range(len(encrypted_message)-1):
        # Convert letters to corresponding entry in the alphabet (a = 0, b = 2, ..., ' ' = 26)
        first_letter = ord(encrypted_message[i])-97
        second_letter = ord(encrypted_message[i+1])-97

        if(first_letter == 32 - 97): #whitespace correpsonds to 32
            first_letter = 26
        if(second_letter == 32 - 97):
            second_letter = 26

        log_lik = log_lik + log_transition_matrix[first_letter, second_letter]
    
    return log_lik

In [9]:
print("log likelihood for encrypted message:", compute_likelihood(encrypted_message, log_matrix))
print("log likelihood for original message:", compute_likelihood(message.lower(), log_matrix)) # ok good the original one has higher probability

log likelihood for encrypted message: 0.9915344182431921
log likelihood for original message: 1.2408537083162372


In [10]:
def explore_space(encrypted_message, log_transition_matrix, alphabet, max_iteration = 10000):
    # print(encrypted_message)

    current_permutation = AlphabetPermutation(alphabet)
    current_permutation.generate_permutation() # start with a random proposal
    # print(current_permutation.describe_permutation())


    # Compute likelihood on original message
    proposed = permutation_generator.permute_string(encrypted_message)
    old_likelihood = compute_likelihood(proposed, log_transition_matrix)

    # store max likelihood achieved during the exploration
    max_reached = compute_likelihood(encrypted_message, log_transition_matrix)
    
    for i in range(max_iteration):
        # Compute the likelihood before the proposal update

        # Choose two letters at random to swap
        k = random.randint(0, 25)
        j = random.randint(0, 25)
        
        #print(permutation_generator.permutation[chr(i + 97)])
        #print(permutation_generator.permutation[chr(j + 97)])


        # To ensure we actually do a swap
        while k == j:
            j = random.randint(0, 25)
        
        
        # Swap the two keys in the permutation class
        permutation_generator.swap_letters(k, j)
        
        #print(permutation_generator.permutation[chr(i + 97)])
        #print(permutation_generator.permutation[chr(j + 97)])

        # Translate the message using this new key and evaluate
        proposed = permutation_generator.permute_string(encrypted_message)
        new_likelihood = compute_likelihood(proposed, log_transition_matrix)

        # print("old", old_likelihood, "new", new_likelihood)


        if new_likelihood > max_reached:
            max_reached = new_likelihood
            best = proposed

        # Compute the acceptance probability in a MH fashion
        acceptance_prob = min(1, np.exp(new_likelihood - old_likelihood))
        #print(new_likelihood, old_likelihood)

        if np.random.uniform() < acceptance_prob: # Accept
            old_likelihood = new_likelihood
        else: # Reject swap
            permutation_generator.swap_letters(k, j) # swap back
        
        if i%(max_iteration/100) == 0:
            print(permutation_generator.permute_string(encrypted_message))
    
    print("The max likelihood we reached was:", max_reached)



In [11]:
encrypted_message = "lrob ive serg cehmttmth lrob ive lmryi bobeti m bag ajboyi yag ol bg afquamtiatfe nmiv gou gour battery mbpreyymth be nmiv ive lujjeyi cejmel ol gour arrohatfe gour fotfemi atx gour yejlmyv xmyxamt ol ive leejmthy ol oivery nere yufv ay io lorb ive hroutxnord ol xmyapprocaimot ot nvmfv yuffeexmth esetiy vase cumji yo mbbosacje a xmyjmde atx m vax toi dtont gou a botiv celore m leji ivai gou nere ive jayi bat mt ive norjx nvob m foujx eser ce presamjex ot io barrg"
explore_space(encrypted_message, log_matrix, alphabet)

dbel vqc wcba mczgiigiz dbel vqc dgbtv lelciv g lpa pjletv tpa ed la puofpgivpiuc sgvq aef aefb lpiicbt glrbcttgiz lc sgvq vqc dfjjctv mcjgcd ed aefb pbbezpiuc aefb ueiucgv piy aefb tcjdgtq ygtypgi ed vqc dccjgizt ed evqcbt scbc tfuq pt ve debl vqc zbefiysebx ed ygtprrbempvgei ei sqguq tfuuccygiz cwcivt qpwc mfgjv te gllewpmjc p ygtjgxc piy g qpy iev xiesi aef p leivq mcdebc g dcjv vqpv aef scbc vqc jptv lpi gi vqc sebjy sqel g uefjy cwcb mc rbcwpgjcy ei ve lpbba
jpor ust ntpc xtdkggkgd jpor ust jkpvu rortgu k ryc yqrovu vyc oj rc yizfykguygit ekus cof cofp ryggtpv krbptvvkgd rt ekus ust jfqqtvu xtqktj oj cofp yppodygit cofp iogitku ygm cofp vtqjkvs mkvmykg oj ust jttqkgdv oj oustpv etpt vfis yv uo jopr ust dpofgmeoph oj mkvybbpoxyukog og eskis vfiittmkgd tntguv synt xfkqu vo krronyxqt y mkvqkht ygm k sym gou hgoeg cof y rogus xtjopt k jtqu usyu cof etpt ust qyvu ryg kg ust eopqm esor k iofqm tntp xt bptnykqtm og uo ryppc
pcdz isf gfct kfqwyywyq pcdz isf pwcni zdzfyi w zmt mazdni nmt d

In [12]:
compute_likelihood("from the very beginning from the first moment i may almost say of my acquaintance with you your manners impressing me with the fullest belief of your arrogance your conceit and your selfish disdain of the feelings of others were such as to form the groundwork of disapprobation on which succeeding events have built so immovable a dislike and i had not known you a month before i felt that you were the last man in the world whom i could ever be prevailed on to marry", log_matrix)

2.232198558062426

In [96]:
np.random.uniform()

0.4576700664195398